# Student Loan Risk with Deep Learning

In [2]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [3]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [4]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [5]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [6]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
print(y.sample(5))

703     0
1440    1
1497    1
1219    1
1246    0
Name: credit_ranking, dtype: int64


In [22]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns=["credit_ranking"])

# Review the features DataFrame
print(X.head())  # Display the first five rows of X

   payment_history  location_parameter  stem_degree_score  gpa_ranking  \
0              7.4                0.70               0.00          1.9   
1              7.8                0.88               0.00          2.6   
2              7.8                0.76               0.04          2.3   
3             11.2                0.28               0.56          1.9   
4              7.4                0.70               0.00          1.9   

   alumni_success  study_major_code  time_to_completion  \
0           0.076              11.0                34.0   
1           0.098              25.0                67.0   
2           0.092              15.0                54.0   
3           0.075              17.0                60.0   
4           0.076              11.0                34.0   

   finance_workshop_score  cohort_ranking  total_loan_score  \
0                  0.9978            3.51              0.56   
1                  0.9968            3.20              0.68   
2          

### Step 3: Split the features and target sets into training and testing datasets.


In [37]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print(f"Training Features Shape: {X_train.shape}")
print(f"Testing Features Shape: {X_test.shape}")
print(f"Training Target Shape: {y_train.shape}")
print(f"Testing Target Shape: {y_test.shape}")

Training Features Shape: (1279, 11)
Testing Features Shape: (320, 11)
Training Target Shape: (1279,)
Testing Target Shape: (320,)


### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [51]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [52]:
# Define the the number of inputs (features) to the model
number_of_features = X_train_scaled.shape[1]

# Review the number of features
print(f"Number of input features: {number_of_features}")

Number of input features: 11


In [64]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = number_of_features * 2  

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = number_of_features  

# Define the number of neurons in the output layer
output_neurons = 1
print(f"Hidden Layer 1 Nodes: {hidden_nodes_layer1}")
print(f"Hidden Layer 2 Nodes: {hidden_nodes_layer2}")
print(f"Output Layer Neurons: {output_neurons}")

Hidden Layer 1 Nodes: 22
Hidden Layer 2 Nodes: 11
Output Layer Neurons: 1


In [76]:
# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer
model.add(Dense(units=hidden_nodes_layer1, activation='relu', input_dim=number_of_features))

# Add the second hidden layer
model.add(Dense(units=hidden_nodes_layer2, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(units=output_neurons, activation='sigmoid'))  # Sigmoid for binary classification

/opt/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [77]:
# Display the Sequential model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 22)             │           264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 11)             │           253 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            12 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 529 (2.07 KB)

 Trainable params: 529 (2.07 KB)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [86]:
# Compile the Sequential model
model.compile(
    loss="binary_crossentropy",  # Binary classification loss function
    optimizer="adam",            # Adam optimizer for efficient training
    metrics=["accuracy"]         # Evaluate performance using accuracy
)


In [87]:
# Fit the model using 50 epochs and the training data
history = model.fit(
    X_train_scaled,  # Training features (scaled)
    y_train,         # Training labels
    epochs=50,       # Number of epochs
    batch_size=32,   # Batch size (default is 32, can be adjusted)
    validation_data=(X_test_scaled, y_test),  # Validation data
    verbose=1        # Displays progress during training
)
print("Model training complete!")


Epoch 1/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6056 - loss: 0.6608 - val_accuracy: 0.6125 - val_loss: 0.6243
Epoch 2/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6926 - loss: 0.5920 - val_accuracy: 0.6687 - val_loss: 0.5791
Epoch 3/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7369 - loss: 0.5546 - val_accuracy: 0.6969 - val_loss: 0.5495
Epoch 4/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7480 - loss: 0.5373 - val_accuracy: 0.7156 - val_loss: 0.5340
Epoch 5/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7495 - loss: 0.5295 - val_accuracy: 0.7188 - val_loss: 0.5251
Epoch 6/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7456 - loss: 0.5191 - val_accuracy: 0.7188 - val_loss: 0.5214
Epoch 7/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7660 - loss: 0.4967 - val_accuracy: 0.7156 - val_loss: 0.5191
Epoch 8/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7685 - loss: 0.4974 - val_accuracy: 0.7188 - val_loss:

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [88]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=1)

# Display the model loss and accuracy results
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7624 - loss: 0.4892 
Test Loss: 0.5233
Test Accuracy: 0.7406


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [89]:
# Set the model's file path
model_filepath = "student_loans.keras"

# Export your model to a keras file
model.save(model_filepath)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [91]:
from tensorflow.keras.models import load_model
# Set the model's file path
model_filepath = "student_loans.keras"

# Load the model to a new object
loaded_model = load_model(model_filepath)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [98]:
# Make predictions with the test data
predictions = loaded_model.predict(X_test_scaled)

# Display a sample of the predictions
print("Sample Predictions (Raw Probabilities):")
print(predictions[:5])

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 998us/step
Sample Predictions (Raw Probabilities):
[[0.31785163]
 [0.34851253]
 [0.9042943 ]
 [0.7511231 ]
 [0.9783492 ]]


In [99]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame((predictions > 0.5).astype(int), columns=["Predictions"])

### Step 4: Display a classification report with the y test data and predictions

In [100]:
# Print the classification report with the y test data and predictions
# Import classification report from sklearn
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(y_test, predictions_df["Predictions"])

# Print the classification report
print(report)


              precision    recall  f1-score   support

           0       0.71      0.77      0.74       154
           1       0.77      0.71      0.74       166

    accuracy                           0.74       320
   macro avg       0.74      0.74      0.74       320
weighted avg       0.74      0.74      0.74       320



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.
To build a recommendation system for student loans, I would collect the following data:

    Student Demographics (Age, Education Level, Degree Program, Expected Graduation Year)
    Financial Status (Income, Family Income, Current Debt, Credit Score)
    Loan History (Previous Loans, Repayment Status, Loan Amount)
    Interest Rate Preferences (Fixed vs. Variable, Preferred Term Length)
    Repayment Ability (Monthly Budget, Employment Status, Expected Salary Post-Graduation)

This data is relevant and appropriate because it helps match students with loans they are likely to qualify for and successfully repay. It ensures personalized recommendations that fit individual financial situations and future earning potential.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.
The recommendation system would primarily use content-based filtering. The system relies on personal financial and demographic data to recommend loans that best fit the student's profile. Unlike collaborative filtering, which depends on user similarity and historical preferences, this system needs to tailor recommendations based on individual eligibility and loan terms. Context-based filtering (a hybrid of content-based and collaborative) could be used if additional behavioral data, such as peer comparisons or career prospects, were included.


3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

Data Privacy and Security
Concern: Student financial and credit information is highly sensitive. Ensuring proper encryption, compliance with data protection laws (e.g., GDPR, CCPA), and preventing unauthorized access is critical.
Impact: A breach could lead to identity theft, financial fraud, and legal consequences for the system provider.

Loan Eligibility and Bias
Concern: If the recommendation system is trained on biased data (e.g., students from wealthier backgrounds getting better loan options), it may unintentionally discriminate against lower-income students.
Impact: This could lead to unfair lending practices and reinforce socioeconomic disparities in education financing.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

